Status dos pedidos abertos

In [1]:
import os
from datetime import datetime
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook

# Pasta onde os novos arquivos serão verificados
pasta_dados = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/4 - MRP/1 - MPS Daily/'

# Arquivo de registro para controlar a data da última execução para cada arquivo
arquivo_log = 'last_run_log.txt'

def verificar_novo_arquivo():
    arquivos = os.listdir(pasta_dados)
    arquivos = [arquivo for arquivo in arquivos if arquivo.startswith('Piloto - MRP_') and arquivo.endswith('.xlsx')]
    if arquivos:
        arquivo_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta_dados, x)))
        return arquivo_recente
    else:
        return None

def carregar_log():
    if os.path.exists(arquivo_log):
        with open(arquivo_log, 'r') as f:
            log = f.read().strip().split('\n')
        log_dict = {linha.split(',')[0]: linha.split(',')[1] for linha in log}
        return log_dict
    else:
        return {}

def salvar_log(log_dict):
    with open(arquivo_log, 'w') as f:
        for arquivo, data in log_dict.items():
            f.write(f"{arquivo},{data}\n")

def verificar_ultima_execucao(arquivo, log_dict):
    return log_dict.get(arquivo, None)

def registrar_ultima_execucao(arquivo, data, log_dict):
    log_dict[arquivo] = data

def processar_arquivo(arquivo, log_dict):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(pasta_dados, arquivo)
    # Ler a aba específica do arquivo Excel
    aba = '8. Entrega Real'
    try:
        df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")
        return

    # Verificar se a coluna 'Status' está presente
    if 'Status' not in df.columns:
        print(f"O arquivo {arquivo} não contém a coluna 'Status'. Ignorando.")
        return

    # Supondo que o nome da coluna é 'Status', mas pode haver espaços extras
    df.columns = df.columns.str.strip()

    # Definir todas as categorias de status desejadas
    categorias_status = ["Ag. Confirmação", "Confirmado", "Parcialmente conf", "Recusado", "Contrato Pendente"]

    # Excluir os valores em branco na coluna 'Status'
    df = df[df['Status'].notna()]

    # Contar a ocorrência de cada status
    contagem_status = df['Status'].value_counts().reindex(categorias_status, fill_value=0)

    # Obter a data atual
    data_atual = datetime.today().date().strftime('%d/%m/%Y')

    # Verificar se o processamento já foi feito hoje para este arquivo
    ultima_execucao = verificar_ultima_execucao(arquivo, log_dict)
    if ultima_execucao == data_atual:
        print(f"O processamento para o arquivo {arquivo} já foi realizado hoje.")
        return

    # Criar um DataFrame final no formato "long" com a data, status e contagem apenas para "Confirmado"
    df_final = pd.DataFrame({
        'data': [data_atual],
        'status': ['Confirmado'],
        'contagem': [contagem_status['Confirmado']],
        'fornecedor': [None]
    })

    # Para as categorias de status exceto "Confirmado", adicionar a lista de fornecedores únicos e suas contagens
    fornecedores_unicos = df[df['Status'] != 'Confirmado'][['Status', 'Fornecedor']]
    contagem_fornecedores = fornecedores_unicos.groupby(['Status', 'Fornecedor']).size().reset_index(name='contagem')

    # Criar um DataFrame com linhas separadas para cada fornecedor com suas respectivas contagens
    fornecedores_separados = []
    for _, row in contagem_fornecedores.iterrows():
        fornecedores_separados.append([data_atual, row['Status'], row['contagem'], row['Fornecedor']])

    df_fornecedores = pd.DataFrame(fornecedores_separados, columns=['data', 'status', 'contagem', 'fornecedor'])

    # Concatenar os DataFrames df_final e df_fornecedores
    df_final_completo = pd.concat([df_final, df_fornecedores], ignore_index=True)

    # Nome do novo arquivo Excel
    diretorio_destino = Path('C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python')
    arquivo_destino = diretorio_destino / 'contagem_status.xlsx'

    # Verificar se o diretório de destino existe, criar se não existir
    diretorio_destino.mkdir(parents=True, exist_ok=True)

    # Carregar o arquivo Excel ou criar um novo DataFrame vazio
    if arquivo_destino.exists():
        # Carregar o arquivo existente
        try:
            wb = load_workbook(arquivo_destino)
            if 'Status' in wb.sheetnames:
                # Carregar o DataFrame da aba 'Status'
                df_anterior = pd.read_excel(arquivo_destino, sheet_name='Status')
                # Mesclar os novos dados com os dados existentes
                df_atualizado = pd.concat([df_anterior, df_final_completo], ignore_index=True)
                # Escrever de volta para a aba 'Status'
                with pd.ExcelWriter(arquivo_destino, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    df_atualizado.to_excel(writer, sheet_name='Status', index=False)
            else:
                # Caso a aba 'Status' não exista, criar uma nova
                with pd.ExcelWriter(arquivo_destino, engine='openpyxl', mode='a') as writer:
                    df_final_completo.to_excel(writer, sheet_name='Status', index=False)
        except Exception as e:
            print(f"Erro ao processar o arquivo {arquivo_destino}: {e}")
    else:
        # Criar um novo arquivo Excel com os dados
        with pd.ExcelWriter(arquivo_destino, engine='openpyxl', mode='w') as writer:
            df_final_completo.to_excel(writer, sheet_name='Status', index=False)

    print(f"Arquivo {arquivo_destino} atualizado com sucesso para o arquivo {arquivo}.")
    registrar_ultima_execucao(arquivo, data_atual, log_dict)

def main():
    log_dict = carregar_log()
    novo_arquivo = verificar_novo_arquivo()
    if not novo_arquivo:
        print("Não há novos arquivos para processar. Encerrando o programa.")
        return

    processar_arquivo(novo_arquivo, log_dict)
    salvar_log(log_dict)

if __name__ == "__main__":
    main()


Arquivo C:\Users\corteda1\Mars Inc\Brazil_MW_PlanningComex - Documents\__INPUT_DATA_FOR_AUTOMATION__\MRP\Python\contagem_status.xlsx atualizado com sucesso para o arquivo Piloto - MRP_01.08 - MPS e Ciclo P07.xlsx.


Overdue PO

In [2]:
import os
from datetime import datetime
import pandas as pd
from openpyxl import load_workbook

# Pasta onde os novos arquivos serão verificados
pasta_dados = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/4 - MRP/1 - MPS Daily/'

# Arquivo de registro para controlar a data da última execução para cada arquivo
arquivo_log = 'last_run_log.txt'

def verificar_novo_arquivo():
    arquivos = os.listdir(pasta_dados)
    arquivos = [arquivo for arquivo in arquivos if arquivo.startswith('Piloto - MRP_') and arquivo.endswith('.xlsx')]
    if arquivos:
        arquivo_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta_dados, x)))
        return arquivo_recente
    else:
        return None

def carregar_log():
    if os.path.exists(arquivo_log):
        with open(arquivo_log, 'r') as f:
            log = f.read().strip().split('\n')
        log_dict = {linha.split(',')[0]: linha.split(',')[1] for linha in log}
        return log_dict
    else:
        return {}

def salvar_log(log_dict):
    with open(arquivo_log, 'w') as f:
        for arquivo, data in log_dict.items():
            f.write(f"{arquivo},{data}\n")

def verificar_ultima_execucao(arquivo, log_dict):
    return log_dict.get(arquivo, None)

def registrar_ultima_execucao(arquivo, data, log_dict):
    log_dict[arquivo] = data

def processar_arquivo(arquivo, log_dict):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(pasta_dados, arquivo)
    # Ler a aba específica do arquivo Excel
    aba = '7. Pedidos'
    try:
        df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")
        return

    # Verificar se a coluna 'Overdue PO' está presente
    if 'Overdue PO' not in df.columns:
        print(f"O arquivo {arquivo} não contém a coluna 'Overdue PO'. Ignorando.")
        return

    # Supondo que o nome da coluna é 'overdue po', mas pode haver espaços extras
    df.columns = df.columns.str.strip()

    # Definir todas as categorias de status desejadas
    categorias_status = ["em dia", "atrasado"]

    # Contar a ocorrência de cada status e filtrar informações adicionais para "atrasado"
    contagem_status = df['Overdue PO'].value_counts().reindex(categorias_status, fill_value=0)
    dados_atrasado = df[df['Overdue PO'] == 'atrasado'][['Texto breve de material', 'Pedido', 'Material', 'DATA REAL']]

    # Obter a data atual
    data_atual = datetime.today().date().strftime('%d/%m/%Y')

    # Criar um DataFrame final no formato "long" com a data, status e contagem
    df_final = pd.DataFrame({
        'data': [data_atual] * len(contagem_status),
        'status': contagem_status.index,
        'contagem': contagem_status.values
    })

    # Adicionar informações adicionais dos pedidos atrasados
    if not dados_atrasado.empty:
        df_atrasado = pd.DataFrame({
            'data': [data_atual] * len(dados_atrasado),
            'DATA REAL': dados_atrasado['DATA REAL'].values,
            'status': ['atrasado'] * len(dados_atrasado),
            'Cod. Material': dados_atrasado['Material'].values,
            'Material': dados_atrasado['Texto breve de material'].values,
            'Número do Pedido': dados_atrasado['Pedido'].values,
            'contagem': [1] * len(dados_atrasado)
        })
        df_final = pd.concat([df_final, df_atrasado], ignore_index=True)

    # Nome do novo arquivo Excel
    arquivo_destino = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx'

    # Carregar o arquivo existente ou criar um novo DataFrame se o arquivo não existir
    if os.path.exists(arquivo_destino):
        # Carregar a planilha existente sem sobrescrever outras abas
        with pd.ExcelFile(arquivo_destino) as reader:
            # Carregar todas as abas existentes em um dicionário
            planilhas = {sheet: pd.read_excel(reader, sheet_name=sheet) for sheet in reader.sheet_names}
        # Atualizar ou criar a aba 'overdue PO'
        if 'overdue PO' in planilhas:
            df_existente = planilhas['overdue PO']
            df_final = pd.concat([df_existente, df_final], ignore_index=True)
        planilhas['overdue PO'] = df_final
    else:
        planilhas = {'overdue PO': df_final}

    # Salvar todas as abas de volta na planilha
    with pd.ExcelWriter(arquivo_destino, engine='openpyxl') as writer:
        for sheet_name, df_sheet in planilhas.items():
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Arquivo {arquivo_destino} atualizado com sucesso para o arquivo {arquivo}.")
    registrar_ultima_execucao(arquivo, data_atual, log_dict)

def main():
    log_dict = carregar_log()
    novo_arquivo = verificar_novo_arquivo()
    if not novo_arquivo:
        print("Não há novos arquivos para processar. Encerrando o programa.")
        return

    processar_arquivo(novo_arquivo, log_dict)

    salvar_log(log_dict)

if __name__ == "__main__":
    main()




Arquivo C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx atualizado com sucesso para o arquivo Piloto - MRP_01.08 - MPS e Ciclo P07.xlsx.


Case fill

In [3]:
import os
from datetime import datetime
import pandas as pd
from openpyxl import load_workbook

# Pasta onde os novos arquivos serão verificados
pasta_dados = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/4 - MRP/1 - MPS Daily/'

# Arquivo de registro para controlar a data da última execução para cada arquivo
arquivo_log = 'last_run_log.txt'

def verificar_novo_arquivo():
    arquivos = os.listdir(pasta_dados)
    arquivos = [arquivo for arquivo in arquivos if arquivo.startswith('Piloto - MRP_') and arquivo.endswith('.xlsx')]
    if arquivos:
        arquivo_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta_dados, x)))
        return arquivo_recente
    else:
        return None

def carregar_log():
    if os.path.exists(arquivo_log):
        with open(arquivo_log, 'r') as f:
            log = f.read().strip().split('\n')
        log_dict = {linha.split(',')[0]: linha.split(',')[1] for linha in log}
        return log_dict
    else:
        return {}

def salvar_log(log_dict):
    with open(arquivo_log, 'w') as f:
        for arquivo, data in log_dict.items():
            f.write(f"{arquivo},{data}\n")

def verificar_ultima_execucao(arquivo, log_dict):
    return log_dict.get(arquivo, None)

def registrar_ultima_execucao(arquivo, data, log_dict):
    log_dict[arquivo] = data

def processar_arquivo(arquivo, log_dict):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(pasta_dados, arquivo)
    # Ler a aba específica do arquivo Excel
    aba = '7. Pedidos'
    try:
        df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")
        return

    # Verificar se a coluna 'Case fill' está presente
    if 'Case fill' not in df.columns:
        print(f"O arquivo {arquivo} não contém a coluna 'Case fill'. Ignorando.")
        return

    # Supondo que o nome da coluna é 'Case fill', mas pode haver espaços extras
    df.columns = df.columns.str.strip()

    # Definir todas as categorias de status desejadas
    categorias_status = ["Not Full", "In full"]

    # Contar a ocorrência de cada status e filtrar informações adicionais para "atrasado"
    contagem_status = df['Case fill'].value_counts().reindex(categorias_status, fill_value=0)
    dados_atrasado = df[df['Case fill'] == 'Not Full'][['Texto breve de material', 'Pedido']]

    # Obter a data atual
    data_atual = datetime.today().date().strftime('%d/%m/%Y')

    # Verificar se o processamento já foi feito hoje para este arquivo
    #ultima_execucao = verificar_ultima_execucao(arquivo, log_dict)
    #if ultima_execucao == data_atual:
    #    print(f"O processamento para o arquivo {arquivo} já foi realizado hoje.")
    #    return

    # Criar um DataFrame final no formato "long" com a data, status e contagem
    df_final = pd.DataFrame({
        'data': [data_atual] * len(contagem_status),
        'status': contagem_status.index,
        'contagem': contagem_status.values
    })

    # Adicionar informações adicionais dos pedidos atrasados
    if not dados_atrasado.empty:
        df_atrasado = pd.DataFrame({
            'data': [data_atual] * len(dados_atrasado),
            'status': ['Not Full'] * len(dados_atrasado),
            'Material': dados_atrasado['Texto breve de material'].values,
            'Número do Pedido': dados_atrasado['Pedido'].values
        })
        df_final = pd.concat([df_final, df_atrasado], ignore_index=True)

    # Nome do novo arquivo Excel
    arquivo_destino = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx'

    # Carregar o arquivo existente ou criar um novo DataFrame se o arquivo não existir
    if os.path.exists(arquivo_destino):
        # Carregar a planilha existente sem sobrescrever outras abas
        with pd.ExcelFile(arquivo_destino) as reader:
            # Carregar todas as abas existentes em um dicionário
            planilhas = {sheet: pd.read_excel(reader, sheet_name=sheet) for sheet in reader.sheet_names}
        # Atualizar ou criar a aba 'Case fill'
        if 'Case fill' in planilhas:
            df_existente = planilhas['Case fill']
            df_final = pd.concat([df_existente, df_final], ignore_index=True)
        planilhas['Case fill'] = df_final
    else:
        planilhas = {'Case fill': df_final}

    # Salvar todas as abas de volta na planilha
    with pd.ExcelWriter(arquivo_destino, engine='openpyxl') as writer:
        for sheet_name, df_sheet in planilhas.items():
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Arquivo {arquivo_destino} atualizado com sucesso para o arquivo {arquivo}.")
    registrar_ultima_execucao(arquivo, data_atual, log_dict)

def main():
    log_dict = carregar_log()
    novo_arquivo = verificar_novo_arquivo()
    if not novo_arquivo:
        print("Não há novos arquivos para processar. Encerrando o programa.")
        return

    processar_arquivo(novo_arquivo, log_dict)

    salvar_log(log_dict)

if __name__ == "__main__":
    main()


Arquivo C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx atualizado com sucesso para o arquivo Piloto - MRP_01.08 - MPS e Ciclo P07.xlsx.


Cash

In [4]:
import os
from datetime import datetime
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook

# Pasta onde os novos arquivos serão verificados
pasta_dados = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/4 - MRP/1 - MPS Daily/'

def verificar_novo_arquivo():
    arquivos = os.listdir(pasta_dados)
    arquivos = [arquivo for arquivo in arquivos if arquivo.startswith('Piloto - MRP_') and arquivo.endswith('.xlsx')]
    if arquivos:
        arquivo_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta_dados, x)))
        return arquivo_recente
    else:
        return None

def processar_arquivo(arquivo):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(pasta_dados, arquivo)
    # Ler a aba específica do arquivo Excel
    aba_stock = '6. Stock'
    try:
        df_stock = pd.read_excel(caminho_arquivo, sheet_name=aba_stock)
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")
        return

    # Verificar se as colunas "Sub-tipo" e "Cash" estão presentes
    if 'Sub-tipo' not in df_stock.columns or 'Cash' not in df_stock.columns:
        print(f"O arquivo {arquivo} não contém as colunas necessárias na aba '6. Stock'. Ignorando.")
        return

    df_stock.columns = df_stock.columns.str.strip()
    df_stock_resumo = df_stock.groupby('Sub-tipo')['Cash'].sum().reset_index()
    df_stock_resumo['data'] = datetime.today().date().strftime('%d/%m/%Y')

    # Nome do novo arquivo Excel
    diretorio_destino = Path('C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python')
    arquivo_destino = diretorio_destino / 'contagem_status.xlsx'

    # Verificar se o diretório de destino existe, criar se não existir
    diretorio_destino.mkdir(parents=True, exist_ok=True)

    if arquivo_destino.exists():
        try:
            with pd.ExcelWriter(arquivo_destino, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                df_stock_resumo.to_excel(writer, sheet_name='Cash', index=False, header=False, startrow=writer.sheets['Cash'].max_row)
        except Exception as e:
            print(f"Erro ao processar o arquivo {arquivo_destino}: {e}")
    else:
        with pd.ExcelWriter(arquivo_destino, engine='openpyxl', mode='w') as writer:
            df_stock_resumo.to_excel(writer, sheet_name='Cash', index=False)

    print(f"Arquivo {arquivo_destino} atualizado com sucesso para o arquivo {arquivo}.")

def main():
    novo_arquivo = verificar_novo_arquivo()
    if not novo_arquivo:
        print("Não há novos arquivos para processar. Encerrando o programa.")
        return

    processar_arquivo(novo_arquivo)

if __name__ == "__main__":
    main()



Arquivo C:\Users\corteda1\Mars Inc\Brazil_MW_PlanningComex - Documents\__INPUT_DATA_FOR_AUTOMATION__\MRP\Python\contagem_status.xlsx atualizado com sucesso para o arquivo Piloto - MRP_01.08 - MPS e Ciclo P07.xlsx.


Past Due PO

In [7]:
import os
from datetime import datetime, timedelta
import pandas as pd
from openpyxl import load_workbook

# Pasta onde os novos arquivos serão verificados
pasta_dados = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/4 - MRP/1 - MPS Daily/'

# Arquivo de registro para controlar a data da última execução para cada arquivo
arquivo_log = 'last_run_log.txt'

def verificar_novo_arquivo():
    arquivos = os.listdir(pasta_dados)
    arquivos = [arquivo for arquivo in arquivos if arquivo.startswith('Piloto - MRP_') and arquivo.endswith('.xlsx')]
    if arquivos:
        arquivo_recente = max(arquivos, key=lambda x: os.path.getmtime(os.path.join(pasta_dados, x)))
        return arquivo_recente
    else:
        return None

def carregar_log():
    if os.path.exists(arquivo_log):
        with open(arquivo_log, 'r') as f:
            log = f.read().strip().split('\n')
        log_dict = {linha.split(',')[0]: linha.split(',')[1] for linha in log}
        return log_dict
    else:
        return {}

def salvar_log(log_dict):
    with open(arquivo_log, 'w') as f:
        for arquivo, data in log_dict.items():
            f.write(f"{arquivo},{data}\n")

def verificar_ultima_execucao(arquivo, log_dict):
    return log_dict.get(arquivo, None)

def registrar_ultima_execucao(arquivo, data, log_dict):
    log_dict[arquivo] = data

def obter_data_util_anterior(data):
    if data.weekday() == 0:  # Segunda-feira
        return data - timedelta(days=3)  # Sexta-feira anterior
    elif data.weekday() == 6:  # Domingo
        return data - timedelta(days=2)  # Sexta-feira anterior
    else:
        return data - timedelta(days=1)  # Dia útil anterior

def processar_arquivo(arquivo, log_dict):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(pasta_dados, arquivo)
    # Ler a aba específica do arquivo Excel
    aba = '7. Pedidos'
    try:
        df = pd.read_excel(caminho_arquivo, sheet_name=aba)
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")
        return

    # Verificar se as colunas necessárias estão presentes
    colunas_necessarias = ['Data documento', 'Material', 'Pedido', 'Past Due PO']
    for coluna in colunas_necessarias:
        if coluna not in df.columns:
            print(f"O arquivo {arquivo} não contém a coluna '{coluna}'. Ignorando.")
            return

    # Filtrar dados pela data útil anterior
    data_hoje = datetime.today().date()
    data_util_anterior = obter_data_util_anterior(data_hoje)
    df['Data documento'] = pd.to_datetime(df['Data documento']).dt.date
    df_filtrado = df[df['Data documento'] == data_util_anterior]

    # Selecionar colunas desejadas
    df_final = df_filtrado[['Data documento', 'Material', 'Pedido', 'Past Due PO']]

    # Nome do novo arquivo Excel
    arquivo_destino = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx'

    # Carregar o arquivo existente ou criar um novo DataFrame se o arquivo não existir
    if os.path.exists(arquivo_destino):
        # Carregar a planilha existente sem sobrescrever outras abas
        with pd.ExcelFile(arquivo_destino) as reader:
            # Carregar todas as abas existentes em um dicionário
            planilhas = {sheet: pd.read_excel(reader, sheet_name=sheet) for sheet in reader.sheet_names}
        # Atualizar ou criar a aba 'Pedidos de ontem'
        if 'Pedidos de ontem' in planilhas:
            df_existente = planilhas['Pedidos de ontem']
            # Concatenar apenas se os DataFrames não estiverem vazios
            if not df_existente.empty and not df_final.empty:
                df_final = pd.concat([df_existente, df_final], ignore_index=True)
            elif df_existente.empty:
                df_final = df_existente
        planilhas['Pedidos de ontem'] = df_final
    else:
        planilhas = {'Pedidos de ontem': df_final}

    # Salvar todas as abas de volta na planilha
    try:
        with pd.ExcelWriter(arquivo_destino, engine='openpyxl') as writer:
            for sheet_name, df_sheet in planilhas.items():
                df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)
    except PermissionError:
        print(f"Erro de permissão ao tentar acessar o arquivo {arquivo_destino}. Verifique se ele não está aberto ou bloqueado.")

    data_atual = data_hoje.strftime('%d/%m/%Y')
    print(f"Arquivo {arquivo_destino} atualizado com sucesso para o arquivo {arquivo}.")
    registrar_ultima_execucao(arquivo, data_atual, log_dict)

def main():
    log_dict = carregar_log()
    novo_arquivo = verificar_novo_arquivo()
    if not novo_arquivo:
        print("Não há novos arquivos para processar. Encerrando o programa.")
        return

    processar_arquivo(novo_arquivo, log_dict)

    salvar_log(log_dict)

if __name__ == "__main__":
    main()



Arquivo C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx atualizado com sucesso para o arquivo Piloto - MRP_01.08 - MPS e Ciclo P07.xlsx.


Recebimentos

In [ ]:
import pandas as pd
from openpyxl import load_workbook, Workbook
import os
import re
from datetime import datetime

# Caminho para o arquivo TXT
txt_file_path = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/SAP/Job MB51_RECEBIMENTOS, Step 1.txt'

# Ler o conteúdo do arquivo
try:
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()
    print("Arquivo TXT lido com sucesso.")
except Exception as e:
    print(f"Erro ao ler o arquivo TXT: {e}")
    raise

# Encontrar a linha que começa com "Pedido"
start_index = None
for i, line in enumerate(lines):
    if "Pedido" in line:
        start_index = i + 1
        break

if start_index is None:
    print("A linha 'Pedido' não foi encontrada no arquivo.")
    raise ValueError("Linha 'Pedido' não encontrada")

# Extrair as linhas de dados a partir da linha "Pedido"
data_lines = lines[start_index:]

# Processar os dados para remover duplicatas e separar por colunas
data = []
for line in data_lines:
    # Verificar se a linha contém números e não é uma linha de separação
    if any(re.search(r'\d', part) for part in line.split('|')) and not re.fullmatch(r'[-\s|]+', line):
        parts = line.strip().split('|')
        if len(parts) > 1:
            data.append(tuple(part.strip() for part in parts if part.strip()))

if not data:
    print("Nenhum dado válido encontrado após a linha 'Pedido'.")
    raise ValueError("Nenhum dado válido encontrado")

# Criar um DataFrame
columns = ['Pedido', 'Item', 'Data lcto', 'Material', 'Qtd']
df = pd.DataFrame(data, columns=columns)

# Substituir vírgulas por pontos e remover pontos de separação de milhar
df['Qtd'] = df['Qtd'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
# Tratar quantidades negativas e valores nulos
df['Qtd'] = df['Qtd'].apply(lambda x: -float(x[:-1]) if pd.notnull(x) and x.endswith('-') else float(x) if pd.notnull(x) else 0.0)

# Remover duplicatas mantendo a primeira ocorrência
df = df.drop_duplicates(subset=['Pedido', 'Item'])

# Somar as quantidades por pedido
summed_df = df.groupby('Pedido', as_index=False).agg({'Qtd': 'sum'})

# Filtrar pedidos com quantidade total diferente de zero
valid_pedidos = summed_df[summed_df['Qtd'] != 0]['Pedido']
filtered_df = df[df['Pedido'].isin(valid_pedidos)].drop(columns=['Qtd'])

# Adicionar coluna com a data atual
filtered_df['Data Processamento'] = datetime.today().strftime('%d/%m/%Y')

print(f"Dados processados e DataFrame filtrado criado com {len(filtered_df)} linhas.")

# Caminho para a planilha Excel de saída
excel_file_path = 'C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx'

# Verificar se o arquivo Excel existe e é válido
if os.path.exists(excel_file_path) and excel_file_path.endswith('.xlsx'):
    try:
        # Carregar a planilha existente
        book = load_workbook(excel_file_path)
        print("Arquivo Excel carregado com sucesso.")
    except Exception as e:
        print(f"Erro ao carregar o arquivo Excel: {e}")
        book = Workbook()
        sheet = book.active
        sheet.title = "recebimento"
        start_row = 1
else:
    book = Workbook()
    sheet = book.active
    sheet.title = "recebimento"
    start_row = 1

# Verificar se a aba 'recebimento' existe e determinar a linha inicial
if 'recebimento' in book.sheetnames:
    sheet = book['recebimento']
    start_row = sheet.max_row + 1
    print(f"Aba 'recebimento' encontrada. Dados serão adicionados a partir da linha {start_row}.")
else:
    sheet = book.create_sheet('recebimento')
    start_row = 1
    print("Aba 'recebimento' criada.")

# Adicionar os dados ao final da aba "recebimento"
for row in filtered_df.itertuples(index=False, name=None):
    sheet.append(row)

# Salvar a planilha
try:
    book.save(excel_file_path)
    # Garantir que a planilha seja editável
    os.chmod(excel_file_path, 0o666)  # Permissões de leitura e escrita para todos
    print(f"Dados adicionados e arquivo Excel salvo: {excel_file_path}")
except Exception as e:
    print(f"Erro ao salvar o arquivo Excel: {e}")
    raise





Arquivo TXT lido com sucesso.
Dados processados e DataFrame filtrado criado com 16 linhas.
Arquivo Excel carregado com sucesso.
Aba 'recebimento' encontrada. Dados serão adicionados a partir da linha 365.
Dados adicionados e arquivo Excel salvo: C:/Users/corteda1/Mars Inc/Brazil_MW_PlanningComex - Documents/__INPUT_DATA_FOR_AUTOMATION__/MRP/Python/contagem_status.xlsx
